# Face Detection and Recognition
This notebook provides a quick resource for exploring OpenCV's built-in face detection and recognition algorithms, based on the third section of the following youtube video: https://www.youtube.com/watch?v=oXlwWbU8l2o

Images of 'famous' people are used as examples, which were downloaded from the Labelled Faces in the Wild (LFW) dataset, available as part of the following Kaggle competition: https://www.kaggle.com/jessicali9530/lfw-dataset

The goal of Face Detection is to identify a rectangular bounding box around each face in an input image. Additional features may be detected as well, e.g. eyes, smile, etc. The goal of Face Recognition is to correctly identify the person based on their facial features. Common practice is to first apply face detection, then apply a facial recognition algorithm to identify the name of the person from features in the detected face image.

## References
- https://github.com/Kaggle/kaggle-api (you can use the api to download the data)
- https://pythonprogramming.net/haar-cascade-object-detection-python-opencv-tutorial/
- https://towardsdatascience.com/a-dog-detector-and-breed-classifier-4feb99e1f852 (also see https://github.com/HenryDashwood/dog_breed_classifier)
- https://keras.io/guides/transfer_learning/
- https://towardsdatascience.com/face-detection-in-2-minutes-using-opencv-python-90f89d7c0f81
- https://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
- 

In [1]:
import cv2 as cv
import numpy as np
import os
import random

In [2]:
dir_root = r"/home/fdpearce/Documents/Projects/data/Images/LFW/lfw-deepfunneled/lfw-deepfunneled"
# Name must match folder name exactly after a single modification: spaces, " ", are converted to underscores, "_"
people = ["Bill Clinton"]
#people = ["George W Bush", "Laura Bush", "Vladimir Putin", "Gloria Macapagal Arroyo", "Arnold Schwarzenegger", "Bill Clinton", \
#          "Hugo Chavez", "Serena Williams", "Colin Powell", "Junichiro Koizumi", "Jennifer Capriati"]

In [3]:
p = [i for i in os.listdir(dir_root)]
print(f"# of People with Images Available = {len(p)}")

# of People with Images Available = 5749


In [4]:
def create_training_data(dir_root, people, person_img_max, face_params, detect_faces=False, random_seed=0, verbose=False):
    """Create training data from the following inputs:
    dir_root = a string containing base directory with subfolders containing images for each person
    people = a list containing string values, with each string containing a person's name. These are the folder names, EXCEPT for spaces instead of underscores
    person_img_max = an integer specifying the maximum number of training images to return for each person. Set this to a value larger than that maximum # of images available if you want to process all images
    face_params = a dictionary containing the parameters to pass to the detectMultiScale method of the haar cascade class, e.g. scaleFactor, minNeighbors
    detect_faces = a boolean-like value (True/False, 1/0, etc) that turns on/off face detection. Turn this off as a dry run to check how much image data will be processed
    random_seed = an integer that determines the order of the random shuffle applied to images before selecting the training samples
    verbose = a boolean-like value that, when truthy, prints additional details during execution for validation/debugging purposes
    """
    random.seed(random_seed)
    features = []
    labels = []
    for person in people:
        img_num = 0
        total_faces = 0
        label = people.index(person)
        path = os.path.join(dir_root, "_".join(person.split(" ")))
        img_files = os.listdir(path)
        random.shuffle(img_files)
        for img in img_files[:min(person_img_max, len(img_files))]:
            img_path = os.path.join(path, img)
            img_num += 1
            print(f"Working on Image # {img_num}: {img}")
            if detect_faces:
                face_feature, face_label = detect_img_faces(img_path, label, verbose=verbose, **face_params)
                if face_label >= 0:
                    total_faces += 1
                    features.append(face_feature)
                    labels.append(face_label)
        print(f"{img_num} training images with {total_faces} faces identified for {person}")
    return features, labels

def detect_primary_face(gray, haar_class_face, verbose=False, max_iter=10, **face_params):
    """Identify the "primary", or most robust, face detected within the input grayscale image, gray.
    """
    num_iter = 0
    face_detected = haar_class_face.detectMultiScale(gray, **face_params)
    num_faces = len(face_detected)
    if verbose:
        print(f"Initial # of Faces = {num_faces}")
    while num_faces != 1 and num_iter != max_iter:
        num_iter += 1
        if verbose:
            print(f"Iteration # = {num_iter}")
        # Update minNeighbors value in copy of params dict
        if num_iter == 1:
            face_params_new = face_params.copy()
        if num_faces == 0 and face_params_new['minNeighbors'] > 1:
            face_params_new['minNeighbors'] -= 1
        elif num_faces > 1:
            face_params_new['minNeighbors'] += 2
        else:
            print("Unable to recognize face in input image")
            print(f"Verify that either 1) num_faces is zero ({num_faces==0}) and minNeighbors is one ({face_params_new['minNeighbors']==1})")
            print(f"OR 2) the maximum # of iterations has been reached ({num_iter==max_iter})")
            print("If either of these scenarios occurs, consider changing the input scaleFactor and/or initial minNeighbors value. If neither 1) or 2) applies, then there is an unknown bug somewhere...")
        if verbose:
            print(f"minNeighbors = {face_params_new['minNeighbors']}")
        face_detected = haar_class_face.detectMultiScale(gray, **face_params_new)
        num_faces = len(face_detected)
    return face_detected

def detect_img_faces(img_path, label, verbose=False, **face_params):
    # Create face detector instance
    # face_params['flags'] = cv.CASCADE_SCALE_IMAGE
    haar_face = "haar_cascade_frontalface.xml" # Path to haar cascade xml file
    haar_class_face = cv.CascadeClassifier(haar_face)
    # Load image and convert to grayscale
    img_array = cv.imread(img_path)
    gray = cv.cvtColor(img_array, cv.COLOR_BGR2GRAY)
    # Detect faces, then store face image as feature and person's name id as label
    #faces_rect = haar_class_face.detectMultiScale(gray, **face_params)
    face_rect = detect_primary_face(gray, haar_class_face, verbose=verbose, **face_params)
    # detected_primary_face returns a list that should only have one face_rect value
    # This is strictly enforced below. Any other face_rect error should be captured by the exception
    # e.g. if face_rect is empty
    try:
        (x, y, w, h) = face_rect[0]
    except Exception as e:
        print(f"The following error occurred when performing face detection for the image at {img_path}:")
        print(e)
        x = None
    if verbose:
        print(*face_rect[0], sep=", ")
    if x:
        face_roi = gray[y:y+h, x:x+w]
        face_label = label
    else:
        face_roi = None
        face_label = -1
    return face_roi, face_label

def show_frame(frame, frame_title):
    cv.imshow(frame_title, frame)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
def draw_show_detected_objects(detected_frame, detected_rect, frame_to_show=None, print_detected=False, rect_color=(255, 255, 255), rect_thickness=2):
    """Print detected object(s) on source image"""
    for i, (x, y, w, h) in enumerate(detected_rect):
        if print_detected:
            print(f"Object {i} Location: x={x}, y={y}, w={w}, h={h}")
        detected_frame = cv.rectangle(detected_frame, (x, y), (x+w, y+h), rect_color, thickness=rect_thickness)
        if isinstance(frame_to_show, np.ndarray):
            show_frame(frame_to_show[y:y+h, x:x+w], "Objects Detected in Image")
    return detected_frame

### Load an Example Image and Convert to Grayscale

In [5]:
person = people[0]
img_num = "0017"
img_ext = ".jpg"

In [7]:
per_fname = "_".join(person.split(" "))
img_path = os.path.join(dir_root, per_fname, "_".join([per_fname, img_num])+img_ext)
img_face = cv.imread(img_path)
gray_face = cv.cvtColor(img_face, cv.COLOR_BGR2GRAY)

## Face Detection

The first step is to download Haar Cascades xml file for frontal face detection from the following link:

https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml

If you want to follow the code below, then you should also download the eye and smile detection xml files from the haarcascades folder.

In [8]:
# scaleFactor = 1.1, minNeighbors = 5 or 10 finds face
# scaleFactor = 1.1, minNeighbors = 5 both eyes and button, or 10 finds right eye and button
# scaleFactor = 1.1, minNeighbors = 100 needed to get the right face w/ no false positives
haar_image = gray_face.copy()
haar_face = 'haar_cascade_frontalface.xml'
haar_eye = 'haar_cascade_eye.xml'
haar_smile = 'haar_cascade_smile.xml'
haar_scaleFactor = 1.2
haar_minNeighbors = 10
haar_minSize = (20, 20)
haar_flags = cv.CASCADE_SCALE_IMAGE
haar_class_face = cv.CascadeClassifier(haar_face)
haar_class_smile = cv.CascadeClassifier(haar_smile)
haar_class_eye = cv.CascadeClassifier(haar_eye)

In [9]:
detect_face_rect = haar_class_face.detectMultiScale(haar_image, scaleFactor=haar_scaleFactor, minNeighbors=haar_minNeighbors, minSize=haar_minSize, flags=haar_flags)
detect_eye_rect = haar_class_eye.detectMultiScale(haar_image, scaleFactor=haar_scaleFactor, minNeighbors=haar_minNeighbors, minSize=haar_minSize, flags=haar_flags)
detect_smile_rect = haar_class_smile.detectMultiScale(haar_image, scaleFactor=haar_scaleFactor, minNeighbors=haar_minNeighbors, minSize=haar_minSize, flags=haar_flags)

In [10]:
print(f"# of Faces Found = {len(detect_face_rect)}")
print(f"# of Eyes Found = {len(detect_eye_rect)}")
print(f"# of Smiles Found = {len(detect_smile_rect)}")

# of Faces Found = 1
# of Eyes Found = 0
# of Smiles Found = 2


In [11]:
# Print detected smile(s) coordinates, and optionally show on original image
haar_image = draw_show_detected_objects(haar_image, detect_smile_rect, print_detected=True, frame_to_show=None, rect_color=(255, 0, 0))

Object 0 Location: x=103, y=148, w=58, h=29
Object 1 Location: x=122, y=96, w=72, h=36


In [12]:
# Print detected eye(s) coordinates, and optionally show on original image
haar_image = draw_show_detected_objects(haar_image, detect_eye_rect, print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))

In [13]:
# Print detected face(s) coordinates, and optionally show on original image
haar_image = draw_show_detected_objects(haar_image, detect_face_rect, print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))

Object 0 Location: x=69, y=66, w=118, h=118


In [14]:
show_frame(haar_image, f"Faces(s) Detected in Image of {person}")

## Face Recognition

### Load Training Data

In [ ]:
person_img_max = 20
detect_face_params = {'scaleFactor': 1.1,
               'minNeighbors': 4,
               'minSize': (20, 20)}
features, labels = create_training_data(dir_root, people, person_img_max, face_params, detect_faces=True, verbose=True)

In [ ]:
print(f"# of Features = {len(features)}")
print(f"# of Labels = {len(labels)}")

In [ ]:
# Visually inspect example images and the bounding box detected by the face recognition algorithm
# Use the verbose setting above to print out bounding boxes
person = people[0]
img_num = ["0017", "0012", "0014", "0029", "0013", "0027", "0020"]
img_ext = ".jpg"
face_rectangles = [[[72, 69, 113, 113]], \
              [[73, 73, 110, 110]], \
              [[61, 61, 126, 126]], \
              [[64, 63, 123, 123]], \
              [[68, 65, 119, 119]], \
              [[70, 65, 118, 118]], \
              [[70, 69, 113, 113]]]

In [ ]:
for ind, im_num in enumerate(img_num):
    per_fname = "_".join(person.split(" "))
    img_path = os.path.join(dir_root, per_fname, "_".join([per_fname, im_num])+img_ext)
    img_face = cv.imread(img_path)
    gray_face = cv.cvtColor(img_face, cv.COLOR_BGR2GRAY)
    gray_face = draw_show_detected_objects(gray_face, face_rectangles[ind], print_detected=True, frame_to_show=None, rect_color=(0, 0, 0))
    show_frame(gray_face, f"Primary Face Detected for {person}")

### 2. Train Face Recognition Model

In [ ]:
save_recognizer_data = False
face_recognizer = cv.face.LBPHFaceRecognizer_create()

In [ ]:
features = np.array(features, dtype="object")
labels = np.array(labels)

In [ ]:
face_recognizer.train(features, labels)

In [ ]:
if save_recognizer_data:
    face_recognizer.save("face_recognizer.yml")
    np.save("features.npy", features)
    np.save("labels.npy", labels)

In [ ]:
tmp=[[]]

In [ ]:
print(*tmp[0])